In [0]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/node-master'       # path to your project on Drive

GIT_USERNAME = "sgsyang" # replace with yours
#GIT_TOKEN = "XXX"           # definitely replace with yours
#GIT_REPOSITORY = "yyy"      # ...nah


drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already   

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
GIT_PATH = "https://github.com/sgsyang/node.git"

!mkdir ./temp
!git clone "{GIT_PATH}"
!mv ./temp/* "{PROJECT_PATH}"
!rm -rf ./temp
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/*  ./

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/node-master’: File exists
Cloning into 'node'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 0), reused 19 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.
mv: cannot stat './temp/*': No such file or directory
sending incremental file list
README.md
          1,555 100%    0.00k

In [0]:
!pip install xgboost
!pip install category_encoders
!pip install tensorboardX
# !pip install qhoptim

%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=0,1
import os, sys
import time
sys.path.insert(0, '..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import lib
import torch, torch.nn as nn
import torch.nn.functional as F
# from qhoptim.pyt import QHAdam


# DATASETS = {
#     'A9A': fetch_A9A,
#     'EPSILON': fetch_EPSILON,
#     'PROTEIN': fetch_PROTEIN,
#     'YEAR': fetch_YEAR,
#     'HIGGS': fetch_HIGGS,
#     'MICROSOFT': fetch_MICROSOFT,
#     'YAHOO': fetch_YAHOO,
#     'CLICK': fetch_CLICK,
# }



device = 'cuda' if torch.cuda.is_available() else 'cpu'

experiment_name = 'CLICK_Xgboost'
experiment_name = '{}_{}.{:0>2d}.{:0>2d}_{:0>2d}:{:0>2d}'.format(experiment_name, *time.gmtime()[:5])
print("experiment:", experiment_name)

     |████████████████████████████████| 81kB 4.0MB/s 
     |████████████████████████████████| 204kB 6.2MB/s 
env: CUDA_VISIBLE_DEVICES=0,1


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


experiment: CLICK_Xgboost_2020.05.26_21:14


In [0]:
torch.cuda.get_device_name(device)

'Tesla P100-PCIE-16GB'

##Default Parameters

In [0]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score



# read the data
data = lib.Dataset("ADULT", random_state=1337, quantile_transform=True, quantile_noise=1e-3)

num_features = data.X_train.shape[1]
num_classes = len(set(data.y_train))


# model = XGBClassifier(objective = 'multi:softmax',num_class = num_classes,tree_method='gpu_hist')
model = XGBClassifier(tree_method='gpu_hist')

model.fit(data.X_train, data.y_train)

pred = model.predict(data.X_test)
# pred = check_numpy(pred)
# pred = np.argmax(logits, axis=1)

accuracy = accuracy_score(data.y_test, pred)
# The classifation Error
# Dataset: ADULT

print('The Error:',1-accuracy)

Unzipping...Done.
The Error: 0.13359950859950864


##Tuned Parameters



In [0]:
!pip install hyperopt
import xgboost as xgb
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [0]:
#AUTOMATE 
import math
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

space={'eta': hp.loguniform('eta',np.exp(-7) ,1),
       'max_depth': hp.quniform("max_depth", 2,10,1),
       'subsample': hp.uniform('subsample', 0.5, 1),
       'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
       'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.5,1),
       'min_child_weight' : hp.loguniform('min_child_weight', np.exp(-16),np.exp(4)),
       'alpha' : hp.choice('alpha', [0,hp.loguniform('a',np.exp(-16),np.exp(2))]),
       'lambda': hp.choice('lambda',[ 0,hp.loguniform('l',np.exp(-16),np.exp(2))]),
       'gamma': hp.choice ('gamma', [0,hp.loguniform('g',np.exp(-16),np.exp(2))]) 
    }

def my_tuning(space):
    reg= XGBClassifier( eta = space['eta'],max_depth = int(space['max_depth']), subsample=space['subsample'],
                         gamma = space['gamma'],reg_alpha = space['alpha'],reg_lambda = space['lambda'],
                         min_child_weight=space['min_child_weight'],
                         colsample_bytree=space['colsample_bytree'],
                         colsample_bylevel = space['colsample_bylevel'],
                         tree_method='gpu_hist')
    
    reg.fit(data.X_train, data.y_train)

    pred = reg.predict(data.X_test)

    accuracy = accuracy_score(data.y_test, pred)
    # The classifation Error
    # Dataset: CLICK 
    # Expected: 0.3461 
    error = 1-accuracy
    print('The Error:',error)
    
    #change the metric if you like
    return {'loss':error, 'status': STATUS_OK }



score_results = [] 
key_results = []

random_state = [1337, 133,13, 1, 602, 295, 970, 2000, 3000, 10000]
# random_state = [1337]
for i in random_state:
  data = lib.Dataset("ADULT", random_state=i, quantile_transform=True, quantile_noise=1e-3)
  print(f'The dataset with random state {i} is used')

  trials = Trials()
  best = fmin(fn=my_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
  # print('Best parameters',best)
  tuned_loss = trials.best_trial['result']['loss']
  print('Best Loss',tuned_loss)
  final = tuned_loss
  print('FINAL Error:',final)
  score_results.append(final)
  key_results.append(i)


import pprint
Result= dict(zip(key_results,score_results))

print('----')
pprint.pprint(Result, width=1)

mean = sum(score_results) / len(score_results)
print('Mean:',mean)
print('Max:',max(score_results))
print('Min:',min(score_results))
print('Distance to max',max(score_results)-mean)
print('Distance to min:',min(score_results)-mean)

# Best Loss 0.12699631449631454
# FINAL mse: 0.12699631449631454
# ----
# {1: 0.12960687960687955,
#  13: 0.12668918918918914,
#  133: 0.12684275184275184,
#  295: 0.12761056511056512,
#  602: 0.12714987714987713,
#  970: 0.1282248157248157,
#  1337: 0.13267813267813267,
#  2000: 0.1277641277641277,
#  3000: 0.12899262899262898,
#  10000: 0.12699631449631454}
# Mean: 0.12825552825552822
# Max: 0.13267813267813267
# Min: 0.12668918918918914
# Distance to max 0.004422604422604448
# Distance to min: -0.001566339066339073

The dataset with random state 1337 is used
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.15202702702702697
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.14588452088452086
The Error:
0.1511056511056511
The Error:
0.15018427518427513
The Error:
0.14204545454545459
The Error:
0.24570024570024573
The Error:
0.24570024570024573
The Error:
0.13636363636363635
The Error:
0.24570024570024573
The Error:
0.13897420147420148
The Error:
0.144502457002457
The Error:
0.24570024570024573

In [0]:
#

# space={'eta': hp.loguniform('eta',math.exp(-7) ,1),
#        'max_depth': hp.quniform("max_depth", 2,10,2),
#        'subsample': hp.uniform('subsample', 0.5, 1),
#        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
#        'colsample_bylevel' : hp.uniform('colsample_bylevel', 0.5,1),
#        'min_child_weight' : hp.loguniform('min_child_weight', math.exp(-16),math.exp(5)),
#        'alpha' : hp.choice('alpha', [0,hp.loguniform('a',math.exp(-16),math.exp(2))]),
#        'lambda': hp.choice('lambda',[ 0,hp.loguniform('l',math.exp(-16),math.exp(2))]),
#        'gamma': hp.choice ('gamma', [0,hp.loguniform('g',math.exp(-16),math.exp(2))])
        
        
#     }


# # Regression: 
# def hyperparameter_tuning(space):
#     reg=xgb.XGBRegressor( eta = space['eta'],max_depth = int(space['max_depth']), subsample=space['subsample'],
#                          gamma = space['gamma'],reg_alpha = space['alpha'],reg_lambda = space['lambda'],
#                         #  min_child_weight=space['min_child_weight'],
#                          colsample_bytree=space['colsample_bytree'],
#                          colsample_bylevel = space['colsample_bylevel'])
    
#     evaluation = [( data.X_train, data.y_train), ( data.X_valid, data.y_valid)]
    
#     reg.fit(data.X_train, data.y_train,
#             eval_set=evaluation, eval_metric="rmse",
#             early_stopping_rounds=10,verbose=False)

#     pred = reg.predict(data.X_valid)
#     mse= mean_squared_error(data.y_valid, pred)
#     print ("SCORE:", mse)
#     #change the metric if you like
#     return {'loss':mse, 'status': STATUS_OK }

In [0]:
# trials = Trials()
# best = fmin(fn=hyperparameter_tuning,
#             space=space,
#             algo=tpe.suggest,
#             max_evals=50,
#             trials=trials)

# print (best)

[16:15:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.6822184
[16:23:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.6917804
[16:29:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.69277877
[16:33:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.6898674
[16:39:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.6913441
[16:46:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
0.68847024
[16:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:


KeyboardInterrupt: ignored